### Imports

In [1]:
%matplotlib inline
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

DATASET_PATH = '../data/train/'
VAL_DATASET_PATH = '../data/val/'
TEST_DATASET_PATH = '../data/test/'

### Create Keras CNN Model

In [2]:
num_classes = 6
im_h, im_w = 224, 224

In [3]:
from keras.backend import clear_session

clear_session()
model = None

Using TensorFlow backend.


In [4]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input as preprocess_input_vgg

from keras.layers import Dense, Dropout, Flatten
from keras.models import Model

In [5]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape = (224,224,3))

In [6]:
flatten = Flatten()(base_model.output)
dropout_1 = Dropout(0.25)(flatten)
fc_1 = Dense(512)(dropout_1)
dropout_2 = Dropout(0.5)(fc_1)
predictions = Dense(num_classes, activation="softmax", name='predictions')(dropout_2)

In [7]:
model = Model(inputs=base_model.input, outputs=predictions)

In [8]:
from keras.losses import categorical_crossentropy
from keras.optimizers import SGD, Adam, Adadelta
from keras.metrics import categorical_accuracy

model.compile(loss=categorical_crossentropy,
              optimizer=SGD(learning_rate=0.01),
              metrics=[categorical_accuracy])

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

### Create Data Iterators

In [9]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator()

In [10]:
BS = 32

train_it = datagen.flow_from_directory(DATASET_PATH, class_mode='categorical', target_size=(im_h,im_w), color_mode='rgb', batch_size=BS)
val_it = datagen.flow_from_directory(VAL_DATASET_PATH, class_mode='categorical', target_size=(im_h,im_w), color_mode='rgb', batch_size=BS)
test_it = datagen.flow_from_directory(TEST_DATASET_PATH, class_mode='categorical', target_size=(im_h,im_w), color_mode='rgb', batch_size=BS)

Found 9000 images belonging to 6 classes.
Found 1350 images belonging to 6 classes.
Found 1800 images belonging to 6 classes.


### Train CNN

In [11]:
nb_train_steps = train_it.samples // train_it.batch_size
nb_val_steps = val_it.samples // val_it.batch_size

In [12]:
from keras.callbacks import EarlyStopping
EPOC = 1

es = EarlyStopping(monitor='val_loss', mode='min', patience=1, verbose=1)

In [13]:
history = model.fit_generator(
    train_it,
    steps_per_epoch=nb_train_steps,
    epochs=EPOC,
    validation_data=val_it,
    validation_steps=nb_val_steps, 
    callbacks=[es]
)

Epoch 1/1
281/281 [==============================] - 293s 1s/step - loss: nan - categorical_accuracy: 0.1679 - val_loss: nan - val_categorical_accuracy: 0.1674


C:\MY_SOFTS\DEV\IDE\MiniConda\envs\ann\lib\site-packages\keras\callbacks\callbacks.py:820: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current - self.min_delta, self.best):


In [ ]:
history.params

In [ ]:
import pandas as pd
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca()
plt.show()

### Check accuracy on test dataset

In [ ]:
model.evaluate(xtest, ytest_cat)

### Check confusion matrix

In [ ]:
import pandas as pd
ypred = model.predict_classes(xtest)

confmatr = pd.crosstab(ytest, ypred, rownames=['True'], colnames=['Predicted'], margins=True)
confmatr

In [ ]:
from sklearn.metrics import confusion_matrix

confmatr = confusion_matrix(ytest, ypred)
row_sums = confmatr.sum(axis=1,keepdims=True)
norm_conf_matr = confmatr / row_sums
np.fill_diagonal(norm_conf_matr,0)
plt.matshow(norm_conf_matr,cmap=plt.cm.gray);

### Save model.

In [ ]:
model.save('../models/model_cnn.h5')

### Try our model on a never seen before picture (original one)

In [ ]:
from keras.models import load_model
mymodel = load_model('../models/model_cnn.h5')

In [ ]:
files = os.listdir(TEST_DATASET_PATH)
idx = random.randint(0,len(files)-1)
origimg = cv2.imread(os.path.join(TEST_DATASET_PATH,files[idx]),cv2.IMREAD_GRAYSCALE)

myimg = origimg/255.
myimg = cv2.resize(myimg,(im_h, im_w))
myimg = myimg.reshape(1,im_h, im_w,1)

myclass = mymodel.predict(myimg)
pred = np.argmax(myclass)

plt.imshow(origimg, cmap='gray');
plt.title('y=%d' % pred);
plt.axis('off');


In [ ]:
plot_6_images(xtest,ypred)